# Lab 2: Azure SRE Agent Basics

**Duration:** 15 minutes  
**Objective:** Configure Azure SRE Agent, connect it to monitoring sources, and interact via natural language

---

## Prerequisites

- Azure subscription with Owner or Contributor access
- Azure resources to monitor (VMs, AKS, App Services, etc.)
- Log Analytics workspace with diagnostic data
- SRE Agent preview access (register at aka.ms/sre-agent)

---

## Architecture Overview

Azure SRE Agent applies the agent primitives you learned in Lab 1 at enterprise scale:

```
┌─────────────────────────────────────────────────────────────────┐
│                     Azure SRE Agent                             │
├─────────────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐          │
│  │    Agent     │  │   Thread     │  │    Run       │          │
│  │  (SRE Core)  │  │ (Incidents)  │  │ (Analysis)   │          │
│  └──────────────┘  └──────────────┘  └──────────────┘          │
│                                                                 │
│  Built-in Tools:                                               │
│  ┌─────────┐ ┌─────────┐ ┌─────────┐ ┌─────────┐ ┌─────────┐  │
│  │Azure    │ │kubectl  │ │az cli   │ │GitHub   │ │Code     │  │
│  │Monitor  │ │         │ │         │ │/ADO     │ │Aware RCA│  │
│  └─────────┘ └─────────┘ └─────────┘ └─────────┘ └─────────┘  │
└─────────────────────────────────────────────────────────────────┘
         │              │              │              │
         ▼              ▼              ▼              ▼
    ┌─────────┐   ┌─────────┐   ┌─────────┐   ┌─────────┐
    │   VMs   │   │   AKS   │   │App Svc  │   │  SQL    │
    └─────────┘   └─────────┘   └─────────┘   └─────────┘
```

---

## Step 1: Environment Setup

Configure the Azure CLI and authenticate.

In [ ]:
# Install dependencies
%pip install python-dotenv --quiet

# Install Azure CLI extensions for SRE Agent
!az extension add --name sre-agent --yes 2>/dev/null || echo "Extension already installed"
!az extension update --name sre-agent 2>/dev/null || echo "Extension up to date"

In [ ]:
import os
import json
import subprocess
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Load environment variables from .env file (shared with Lab 1)
load_dotenv()

# Configuration - loaded from .env file
SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID", "<your-subscription-id>")
RESOURCE_GROUP = os.getenv("AZURE_RESOURCE_GROUP", "rg-sre-workshop")
LOCATION = "eastus"  # SRE Agent preview regions
SRE_AGENT_NAME = "sre-agent-workshop"
LOG_ANALYTICS_WORKSPACE = "law-sre-workshop"

def run_az(cmd):
    """Execute Azure CLI command and return JSON result"""
    result = subprocess.run(
        f"az {cmd} -o json",
        shell=True,
        capture_output=True,
        text=True
    )
    if result.returncode != 0:
        print(f"Error: {result.stderr}")
        return None
    return json.loads(result.stdout) if result.stdout.strip() else None

---

## Step 2: Create SRE Agent Instance

Create an SRE Agent instance connected to your Log Analytics workspace.

In [ ]:
# Ensure resource group exists
run_az(f"group create --name {RESOURCE_GROUP} --location {LOCATION}")

# Create Log Analytics workspace (if not exists)
law = run_az(f"""monitor log-analytics workspace create \
    --resource-group {RESOURCE_GROUP} \
    --workspace-name {LOG_ANALYTICS_WORKSPACE} \
    --location {LOCATION}""")

if law:
    print(f"✓ Log Analytics workspace: {law.get('name')}")
    LAW_ID = law.get('id')
else:
    # Get existing workspace
    law = run_az(f"monitor log-analytics workspace show --resource-group {RESOURCE_GROUP} --workspace-name {LOG_ANALYTICS_WORKSPACE}")
    LAW_ID = law.get('id')
    print(f"✓ Using existing workspace: {LAW_ID}")

In [ ]:
# Create the SRE Agent
# Note: Actual CLI syntax may vary based on preview version

sre_agent_config = {
    "name": SRE_AGENT_NAME,
    "resourceGroup": RESOURCE_GROUP,
    "location": LOCATION,
    "dataSources": {
        "logAnalytics": {
            "workspaceId": LAW_ID
        }
    },
    "capabilities": {
        "anomalyDetection": True,
        "rootCauseAnalysis": True,
        "incidentReports": True,
        "codeAwareRCA": False  # Enable if GitHub/ADO connected
    }
}

print("SRE Agent Configuration:")
print(json.dumps(sre_agent_config, indent=2))

# In production, you would run:
# az sre-agent create --name {name} --resource-group {rg} --config {config.json}
print("\n⚠ Note: Use Azure Portal or CLI to create the SRE Agent with the above config")

---

## Step 3: Configure Monitoring Scope

Define which resources the SRE Agent should monitor.

In [ ]:
# List resources in the subscription that can be monitored
resources = run_az(f"resource list --resource-group {RESOURCE_GROUP}")

if resources:
    print("Resources available for monitoring:")
    print("-" * 50)
    for r in resources[:10]:  # Show first 10
        print(f"  {r.get('type', 'Unknown'):40} {r.get('name')}")
else:
    print("No resources found in resource group")

# Configure monitoring scope
monitoring_scope = {
    "includeResourceGroups": [RESOURCE_GROUP],
    "resourceTypes": [
        "Microsoft.Compute/virtualMachines",
        "Microsoft.ContainerService/managedClusters",
        "Microsoft.Web/sites",
        "Microsoft.Sql/servers"
    ],
    "excludeResources": []  # Specific resource IDs to exclude
}

print("\nMonitoring scope configured:")
print(json.dumps(monitoring_scope, indent=2))

---

## Step 4: Interact via Natural Language

The SRE Agent exposes a natural language interface. Let's simulate interactions.

In [ ]:
# Simulated SRE Agent interaction class
# In production, this would use the actual SRE Agent API

class SREAgentSimulator:
    """
    Simulates Azure SRE Agent interactions for workshop purposes.
    Production code would use the actual SRE Agent REST API or SDK.
    """
    
    def __init__(self, agent_name):
        self.agent_name = agent_name
        self.conversation_history = []
        
        # Simulated anomaly data
        self.anomalies = [
            {
                "resource": "vm-prod-web-01",
                "type": "Microsoft.Compute/virtualMachines",
                "severity": "High",
                "metric": "CPU Percentage",
                "value": 94.5,
                "threshold": 80,
                "detected_at": datetime.now() - timedelta(minutes=15),
                "possible_causes": [
                    "Memory leak in web application process",
                    "Increased traffic spike",
                    "Background job consuming resources"
                ]
            },
            {
                "resource": "aks-prod-cluster",
                "type": "Microsoft.ContainerService/managedClusters",
                "severity": "Medium",
                "metric": "Pod Restart Count",
                "value": 12,
                "threshold": 5,
                "detected_at": datetime.now() - timedelta(minutes=45),
                "possible_causes": [
                    "OOM kills in payment-service pods",
                    "Liveness probe failures",
                    "Node resource pressure"
                ]
            }
        ]
    
    def query(self, user_input: str) -> str:
        """Process a natural language query"""
        self.conversation_history.append({"role": "user", "content": user_input})
        
        lower_input = user_input.lower()
        
        if "status" in lower_input or "health" in lower_input:
            response = self._get_health_status()
        elif "anomal" in lower_input or "issue" in lower_input or "problem" in lower_input:
            response = self._get_anomalies()
        elif "investigate" in lower_input or "diagnose" in lower_input or "analyze" in lower_input:
            response = self._investigate_issue(user_input)
        elif "recommend" in lower_input or "fix" in lower_input or "remediat" in lower_input:
            response = self._get_recommendations()
        else:
            response = self._general_response(user_input)
        
        self.conversation_history.append({"role": "assistant", "content": response})
        return response
    
    def _get_health_status(self) -> str:
        return """## Infrastructure Health Status

| Resource Group | Resources | Healthy | Warning | Critical |
|----------------|-----------|---------|---------|----------|
| rg-sre-workshop | 12 | 9 | 2 | 1 |

**Active Alerts:** 2
- 🔴 **Critical:** vm-prod-web-01 CPU at 94.5% (15 min ago)
- 🟡 **Warning:** aks-prod-cluster pod restarts elevated (45 min ago)

Would you like me to investigate any of these issues?"""
    
    def _get_anomalies(self) -> str:
        anomaly_text = "## Detected Anomalies\n\n"
        for a in self.anomalies:
            severity_icon = "🔴" if a['severity'] == "High" else "🟡"
            anomaly_text += f"""{severity_icon} **{a['resource']}** ({a['type'].split('/')[-1]})
   - Metric: {a['metric']} = {a['value']} (threshold: {a['threshold']})
   - Detected: {a['detected_at'].strftime('%H:%M:%S')}
   - Possible causes:
     - {chr(10).join('     - ' + c for c in a['possible_causes'])}

"""
        return anomaly_text
    
    def _investigate_issue(self, query: str) -> str:
        return """## Investigation Report: vm-prod-web-01 High CPU

**Timeline Analysis:**
- 14:30 - CPU began climbing from baseline 35%
- 14:45 - Crossed 80% threshold, alert triggered
- 15:00 - Peak at 94.5%, sustained

**Correlated Events:**
1. Deployment `webapp-v2.3.1` at 14:28
2. Traffic spike +40% at 14:32 (normal for this time)
3. Memory usage also elevated (78%)

**Log Analysis (top errors):**
```
[ERROR] OutOfMemoryException in PaymentProcessor.ProcessBatch()
[WARN]  Thread pool exhaustion detected
[ERROR] Database connection timeout (3 occurrences)
```

**Root Cause Assessment:**
High confidence (87%): Memory leak introduced in webapp-v2.3.1
The PaymentProcessor.ProcessBatch() method appears to not be disposing 
database connections properly, causing thread pool exhaustion.

**Recommended Actions:**
1. Rollback to webapp-v2.3.0 (immediate mitigation)
2. Scale out to 3 instances (reduce per-instance load)
3. Review PR #1234 for connection disposal fix

Shall I execute any of these remediations?"""
    
    def _get_recommendations(self) -> str:
        return """## Recommended Remediations

### Immediate Actions (< 5 min)
1. **Restart vm-prod-web-01 web service**
   ```bash
   az vm run-command invoke --name vm-prod-web-01 \
     --command-id RunShellScript \
     --scripts "systemctl restart webapp"
   ```

2. **Scale AKS node pool**
   ```bash
   az aks nodepool scale --cluster-name aks-prod-cluster \
     --name nodepool1 --node-count 5
   ```

### Short-term Actions (< 1 hour)
- Deploy hotfix PR #1234 (connection disposal fix)
- Increase VM size from Standard_D2s_v3 to Standard_D4s_v3

### Long-term Actions
- Implement connection pooling
- Add memory pressure alerts at 70% threshold
- Review batch processing architecture

Do you want me to execute any of these commands?"""
    
    def _general_response(self, query: str) -> str:
        return f"""I can help you with:
- **Health status**: "What's the current health status?"
- **Anomaly detection**: "Show me any anomalies or issues"
- **Investigation**: "Investigate the CPU issue on vm-prod-web-01"
- **Recommendations**: "What do you recommend to fix this?"

What would you like to know about your infrastructure?"""

# Initialize the simulator
sre = SREAgentSimulator(SRE_AGENT_NAME)
print(f"✓ SRE Agent '{SRE_AGENT_NAME}' ready for queries")

In [ ]:
# Interactive query: Check health status
print("Query: What's the current health status?")
print("=" * 50)
response = sre.query("What's the current health status?")
print(response)

In [ ]:
# Interactive query: Show anomalies
print("Query: Show me any anomalies")
print("=" * 50)
response = sre.query("Show me any anomalies")
print(response)

In [ ]:
# Interactive query: Investigate an issue
print("Query: Investigate the CPU issue on vm-prod-web-01")
print("=" * 50)
response = sre.query("Investigate the CPU issue on vm-prod-web-01")
print(response)

In [ ]:
# Interactive query: Get recommendations
print("Query: What do you recommend to fix this?")
print("=" * 50)
response = sre.query("What do you recommend to fix this?")
print(response)

---

## Step 5: Generate Incident Report

SRE Agent can generate incident summary reports for documentation and handoff.

In [ ]:
# Generate an incident report
incident_report = {
    "incident_id": "INC-2026-0130-001",
    "title": "High CPU on vm-prod-web-01",
    "severity": "High",
    "status": "Investigating",
    "detected_at": "2026-01-30T14:45:00Z",
    "affected_resources": ["vm-prod-web-01"],
    "impact": "Potential degraded performance for web application users",
    "timeline": [
        {"time": "14:28", "event": "Deployment webapp-v2.3.1 completed"},
        {"time": "14:30", "event": "CPU began climbing from baseline"},
        {"time": "14:45", "event": "Alert triggered - CPU > 80%"},
        {"time": "15:00", "event": "Investigation started by SRE Agent"},
    ],
    "root_cause": {
        "confidence": 0.87,
        "description": "Memory leak in PaymentProcessor.ProcessBatch() - connections not disposed",
        "related_commit": "abc123f",
        "related_pr": "#1234"
    },
    "actions_taken": [],
    "recommended_actions": [
        "Rollback to webapp-v2.3.0",
        "Scale out to 3 instances",
        "Deploy connection disposal fix"
    ]
}

print("=" * 60)
print("INCIDENT REPORT")
print("=" * 60)
print(json.dumps(incident_report, indent=2))

In [ ]:
# Format as markdown for GitHub/ADO issue
markdown_report = f"""
# {incident_report['incident_id']}: {incident_report['title']}

**Severity:** {incident_report['severity']} | **Status:** {incident_report['status']}

## Impact
{incident_report['impact']}

## Timeline
| Time | Event |
|------|-------|
{''.join(f"| {t['time']} | {t['event']} |" + chr(10) for t in incident_report['timeline'])}

## Root Cause Analysis
**Confidence:** {incident_report['root_cause']['confidence']*100:.0f}%

{incident_report['root_cause']['description']}

Related: PR {incident_report['root_cause']['related_pr']} (commit {incident_report['root_cause']['related_commit']})

## Recommended Actions
{''.join('- [ ] ' + a + chr(10) for a in incident_report['recommended_actions'])}

---
*Generated by Azure SRE Agent at {datetime.now().isoformat()}*
"""

print(markdown_report)

---

## Step 6: SRE Agent Memory System

SRE Agent learns from past incidents to improve future investigations.

In [ ]:
# Demonstrate the memory system concept
memory_entry = {
    "pattern": "High CPU after deployment",
    "symptoms": ["CPU > 80%", "Memory climbing", "Thread pool exhaustion"],
    "common_causes": [
        {"cause": "Memory leak", "frequency": 0.45},
        {"cause": "Inefficient query", "frequency": 0.25},
        {"cause": "Connection leak", "frequency": 0.20},
        {"cause": "Infinite loop", "frequency": 0.10}
    ],
    "successful_remediations": [
        {"action": "Rollback deployment", "success_rate": 0.92},
        {"action": "Restart service", "success_rate": 0.78},
        {"action": "Scale out", "success_rate": 0.65}
    ],
    "related_runbooks": [
        "runbook://high-cpu-investigation",
        "runbook://deployment-rollback"
    ]
}

print("SRE Agent Memory Entry:")
print("=" * 50)
print(f"Pattern: {memory_entry['pattern']}")
print(f"\nCommon Causes (by frequency):")
for cause in memory_entry['common_causes']:
    bar = "█" * int(cause['frequency'] * 20)
    print(f"  {cause['cause']:25} {bar} {cause['frequency']*100:.0f}%")

print(f"\nSuccessful Remediations:")
for rem in memory_entry['successful_remediations']:
    print(f"  {rem['action']:25} Success rate: {rem['success_rate']*100:.0f}%")

---

## Summary: SRE Agent Capabilities

| Capability | Description |
|------------|-------------|
| **Anomaly Detection** | Continuous monitoring with ML-based anomaly detection |
| **Natural Language** | Query infrastructure status conversationally |
| **Root Cause Analysis** | Correlate logs, metrics, and deployments |
| **Code-Aware RCA** | Link issues to source code via GitHub/ADO |
| **Incident Reports** | Auto-generate documentation for handoff |
| **Memory System** | Learn from past incidents to improve accuracy |

---

## Next Steps

Proceed to **Lab 3** to learn how to extend SRE Agent with custom sub-agents and MCP servers.

---

## References

| Resource | URL |
|----------|-----|
| Azure SRE Agent Overview | https://learn.microsoft.com/azure/sre-agent/overview |
| SRE Agent Product Page | https://azure.microsoft.com/products/sre-agent |
| SRE Agent Usage Guide | https://learn.microsoft.com/azure/sre-agent/usage |
| Memory System Blog | https://techcommunity.microsoft.com/blog/appsonazureblog/never-explain-context-twice-introducing-azure-sre-agent-memory/4473059 |
| Reimagining AI Ops Blog | https://techcommunity.microsoft.com/blog/appsonazureblog/reimagining-ai-ops-with-azure-sre-agent-new-automation-integration-and-extensibi/4462613 |
| Stop Running Runbooks at 3am | https://techcommunity.microsoft.com/blog/appsonazureblog/stop-running-runbooks-at-3-am-let-azure-sre-agent-do-your-on-call-grunt-work/4479811 |